In [1]:
import pandas as pd
import numpy as np
from IPython.display import display

### <font color="orange">データの読み込み</font>

In [2]:
import zipfile
import os
# 解凍したいzipファイルのパス
zip_file_path = '../../../../0_original_datasets/datasets.zip'

# 解凍先のディレクトリ
folder_path = '../../../tmp_data_folder/0_tmp_original_data/'

if os.path.exists(folder_path):
    print("既に解凍されたフォルダが存在します。")
    pass
else:
    # Zipファイルを開いて解凍する
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(folder_path)

    print(f'{zip_file_path} を {folder_path} に解凍しました。')

既に解凍されたフォルダが存在します。


In [3]:
olist_order_reviews_dataset = pd.read_csv("../../../tmp_data_folder/0_tmp_original_data/datasets/olist_order_reviews_dataset.csv",
                                          dtype={'review_comment_title': 'str'})

### <font color="orange">データの確認</font>

In [4]:
olist_order_reviews_dataset[olist_order_reviews_dataset.duplicated(subset="order_id", keep="first")].shape[0]
#* order_idで551件の重複有り。

551

In [5]:
olist_order_reviews_dataset[olist_order_reviews_dataset.duplicated(subset="order_id", keep=False)].sort_values(by=["order_id", "review_answer_timestamp"]).head(6)

#* ユニークなorder_idに対し、review_idが複数存在するものがある。
#* 本来主キーであるorder_id１件につき、レビュー１件である。
#* レビューの更新等が考えられるため（仮説）、review_answer_timestampが最新のものを採択する。

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
22423,2a74b0559eb58fc1ff842ecc999594cb,0035246a40f520710769010f752e7507,5,NaN,Estou acostumada a comprar produtos pelo barat...,2017-08-25 00:00:00,2017-08-29 21:45:57
25612,89a02c45c340aeeb1354a24e7d4b2c1e,0035246a40f520710769010f752e7507,5,NaN,NaN,2017-08-29 00:00:00,2017-08-30 01:59:12
22779,ab30810c29da5da8045216f0f62652a2,013056cfe49763c6f66bda03396c5ee3,5,NaN,NaN,2018-02-22 00:00:00,2018-02-23 12:12:30
68633,73413b847f63e02bc752b364f6d05ee9,013056cfe49763c6f66bda03396c5ee3,4,NaN,NaN,2018-03-04 00:00:00,2018-03-05 17:02:00
854,830636803620cdf8b6ffaf1b2f6e92b2,0176a6846bcb3b0d3aa3116a9a768597,5,NaN,NaN,2017-12-30 00:00:00,2018-01-02 10:54:06
83224,d8e8c42271c8fb67b9dad95d98c8ff80,0176a6846bcb3b0d3aa3116a9a768597,5,NaN,NaN,2017-12-30 00:00:00,2018-01-02 10:54:47


### <font color="orange">order_idが重複する場合、タイムスタンプが最新のデータのみを抽出。</font>

In [6]:
unique_olist_review_dataset = olist_order_reviews_dataset.copy()
unique_olist_review_dataset["review_answer_timestamp"] = pd.to_datetime(unique_olist_review_dataset["review_answer_timestamp"])
unique_olist_review_dataset = unique_olist_review_dataset.sort_values(by="review_answer_timestamp", ascending=False).drop_duplicates("order_id")

In [7]:
# 確認
unique_olist_review_dataset[unique_olist_review_dataset.duplicated(subset="order_id", keep=False)].sort_values(by=["order_id", "review_answer_timestamp"]).shape[0]

0

### <font color="orange">データの保存</font>

In [8]:
import os

save_path = "../../../tmp_data_folder/1_custom_data/"

# フォルダが存在しない場合に作成
if not os.path.exists(save_path):
    os.makedirs(save_path)
    print(f'フォルダ {save_path} を作成しました。')
else:
    print(f'フォルダ {save_path} は既に存在します。')
    
unique_olist_review_dataset.to_csv("../../../tmp_data_folder/1_custom_data/unique_olist_review_dataset.csv", index=False)
print("../tmp/にunique_olist_review_dataset.csvを保存しました。")

フォルダ ../../../tmp_data_folder/1_custom_data/ は既に存在します。
../tmp/にunique_olist_review_dataset.csvを保存しました。
